In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import joblib
import pandas as pd

# Load model
model = joblib.load("trained_model.pkl")

# Load new data (simulate with X_test)
X_new = pd.read_csv("X_test.csv")
preds = model.predict(X_new)
results = X_new.copy()
results['predicted_breach'] = preds

# Filter true breaches
true_breaches = results[results['predicted_breach'] == 1]

if not true_breaches.empty:
    # Email content
    msg = MIMEMultipart()
    msg['From'] = "your_email@gmail.com"
    msg['To'] = "recipient_email@gmail.com"
    msg['Subject'] = f"Regulatory Breach Alert - {len(true_breaches)} Issues Found"

    body = """<html><body><p>🚨 Breaches Detected:</p>""" + true_breaches.to_html(index=False) + "</body></html>"
    msg.attach(MIMEText(body, 'html'))

    # Send via Gmail SMTP
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("your_email@gmail.com", "your_app_password")
    server.send_message(msg)
    server.quit()
else:
    print("No breaches found. Email not sent.")